## useful model building.

In [2]:
import tensorflow as tf
slim=tf.contrib.slim

LOGITS_SCOPE_NAME = 'logits'
MERGED_LOGITS_SCOPE = 'merged_logits'
IMAGE_POOLING_SCOPE = 'image_pooling'
ASPP_SCOPE = 'aspp'
CONCAT_PROJECTION_SCOPE = 'concat_projection'
DECODER_SCOPE = 'decoder'
def cal_scaled_dim_val(dim,scale_coeff):
    """利用scale_coeff对dim维做成scale的维度.
       这里只是一个计算,计算scale之后的维度数.其实并没有实际scale数据Tensor对象.
    """
    if isinstance(dim,tf.Tensor):
        return tf.cast((tf.tofloat(dim)-1.0)*scale_coeff+1.0,tf.int32) # 其实这里边的这个+1.0是为了向上取整
    else:
        return (float(dim)-1.0)*scale_coeff+1.0

def extract_features(features,
                     model_options,
                     weight_decay=0.0001,
                     reuse=None,
                     is_training=False,
                     fine_tune_batch_norm=False):
    """提取特征图和end_point.
    """
    # 提取
    features,end_points=local_extract_features(
        images,
        output_stride=model_options.output_stride,
        multi_grid=model_options.multi_grid,
        model_variant=model_options.model_variant,
        depth_multiplier=model_options.depth_multiplier,
        weight_decay=weight_decay,
        reuse=reuse,
        is_training=in_training,
        fine_tune_batch_norm=fine_tune_batch_norm)
    
    if not model_options.aspp_with_batch_norm:
        # aspp不需要batch norm,直接返回features
        # 我们知道batch norm是在激活函数之前,对features做的.让其归一化到0~1之间.
        return features,end_points
    else:
        batch_norm_params={
            'is_training':is_training and fine_tune_batch_norm,
            'decay':0.9997,
            'spsilon':1e-5,
            'scale':True,
        }
    # slim.arg_scope对给定的op存储其param
    # 构建figure5里边的 Block4之后处理的ASPP部分.
    with slim.arg_scope(
        [slim.cov2d,slim.separable_conv2d],
        weights_regularizer=slim.l2_regularizer(weight_decay),
        activation_fn=tf.nn.relu,
        normalizer_fn=slim.batch_norm,
        padding='SAME',
        stride=1,
        reuse=reuse):
        # 目的是要做batch norm
        with slim.arg_scope(
            [slim.batch_norm],
            **batch_norm_params):
            depth=256
            branch_logits=[]
            # step 1 做一个pooling
            # 融合B部分(image pooling).
            if model_options.add_image_level_feature:
                
                if model_options.crop_size is not None:
                    image_pooling_crop_size=model_options.image_pooling_crop_size
                    if image_pooling_crop_size is None:
                        image_pooling_crop_size=model_options.crop_size
                    # 计算pooling的scale
                    pooling_height=cal_scaled_dim_val(image_pooling_crop_size[0],
                                                      1.0/model_options.output_stride)
                    pooling_width=cal_scaled_dim_val(image_pooling_crop_size[1],
                                                    1.0/model_options.output_stride)
                    # 加pooling层
                    image_feature_pooled=slim.avg_pool2d(features,
                                    [pooling_height,pooling_width],
                                    [1,1],
                                    padding='VALID')
                    # crop size 也需要做一下scale
                    resize_height=cal_scaled_dim_val(model_options.crop_size[0],
                                                    1.0/model_options.output_stride)
                    resize_width=cal_scaled_dim_val(model_ooptions.crop_size[1],
                                                   1.0/model_options.output_stride)
                else:
                    # 没有crop的size,做一个global的pooling
                    pooling_height=tf.shape(features)[0]
                    pooling_width=tf.shape(features)[1]
                    image_feature_pooled=tf.reduce_mean(
                        features,
                        axis=[1,2])[:,tf.newaxis,tf.newaxis] # 在features基础上再添加两个维度,但是这两个维度还没有其他的填充值.
                    resize_height=pooling_height
                    resize_width=pooling_width
                # 添加一个1x1的卷积
                image_feature=slim.conv2d(
                    image_feature_pooled,depth,1,scope=IMAGE_POOLING_SCOPE)
                # 插值成resize的feature map
                image_feature=tf.image.resize_bilinear(image_feature,[resize_height,resize_width],
                                                      align_corners=True)
                '''
                if isinstance(resize_height,tf.Tensor):
                    resize_height=None
                if isinstance(resize_width,tf.Tensor):
                    resize_width=None
                '''
                image_feature.set_shape([None,resize_height,resize_width,depth])
                branch_logits.append(image_feature)
            
            # step 2 对features做1x1卷积,注意此处并不是对经过pooling的image_feature做1x1卷积.
            # 融合A部分(ASPP) 需要1x1
            temp=slim.conv2d(features,depth,1,scope=ASPP_SCOPE+str(0))
            branch_logits.append(temp)
            
            # ASPP,的金字塔每层采用不同的atrous rates,此处构建这组atrous pyramid
            # 融合A部分(ASPP) 需要3x3 带artous.
            if model_option.atrous_rates:
                # 3x3卷积
                for i,rate in enumerate(model_options.atrous_rates,1):
                    scope=ASPP_SCOPE+str(i)
                    # 如果采用可分离卷积
                    if model_options.aspp_with_separable_conv:
                        aspp_features=split_separable_conv2d(
                            features,
                            filters=depth,
                            rate=rate,
                            weight_decay=weight_decay,
                            scope=scope)
                    else:
                        aspp_features=slim.conv2(features,depth,3,rate=rate,scope=scope)
                    
                    branch_logits.append(aspp_features)
             
            # 把这些组件组合起来
            concat_logits=tf.concat(branch_logits,3)
            concat_logits=slim.conv2d(
                concat_logits,depth,1,scope=CONCAT_PROJECTION_SCOPE)
            concat_logits=slim.dropout(concat_logits,keep_prob=0.9,is_training=is_training,
                                      scope=CONCAT_PROJECTION_SCOPE+'_dropout')
            
                
                    
                    
                
                    
                
            
        
    


def _get_logits(images,model_options,weight_decay=0.0001,reuse=None,is_training=False,
               find_tune_batch_norm=False):
    """生成logits网络.该网络应用到aspp,atrous spatial pyramid pooling.
    """
    # 提取features和end_points.
    features,end_points=extract_features(
        images,
        model_options,
        weight_decay=weight_decay,
        reuse=reuse,
        is_training=is_training,
        find_tune_batch_norm=find_tune_batch_norm)
    
    # 如果decoder 有特殊定义的stride.需要对decoder size做scale
    if model_option.decoder_output_stride is not None:
        if model_option.crop_size is None:
            height=tf.shape(images)[1]
            width=tf.shape(images)[2]
        else:
            # crop存在
            height,width=model_option.crop_size
        
        # 求decoder使用的size.这个是经过decoder_output_stride之后的.
        decoder_height=cal_scaled_dim_val(height,1.0/model_options.decoder_output_stride)
        decoder_width=cal_scaled_dim_val(wid,1.0/model_options.decoder_output_stride)
        
        # 对features做重新refine
        features=refine_decoder(
            features,
            end_points,
            decoder_height=decoder_height,
            decoder_widht=decoder_width,
            decoder_use_separable_conv=model_options.decoder_use_separable_conv, # 使用离散卷积
            model_variant=model_options.model_variant,
            weight_decay=weight_decay,
            reuse=reuse,
            is_training=is_training,
            fine_tune_batch_norm=find_tune_batch_norm)

    # 获得batch的logits
    # batch_normalization作用:
    # 在激活函数之前的bn模块,它接受wx+b计算的feature作为输入.可以做到如下几点:
    # 1. 提高梯度传播数度,将所有输出归一化到0~1.避免梯度消失.
    # 2. 提高模型的收敛速度.(归一化到0~1,所有的feature都是)
    # 3. 减少模型对参数初始化的影响.(归一化到0~1)
    outputs_to_logits={}
    for output in sorted(model_options.outputs_to_num_classes):
        outputs_to_logits[output]=get_batch_logits(
            features,
            model_options.outputs_to_num_classes[output],
            model_options.atrous_rates,
            aspp_with_batch_norm=model_options.aspp_with_batch_norm, # batch normalization
            weight_decay=weight_decay,
            reuse=reuse,
            scope_suffix=output) # scope_suffix后缀
        
    return outputs_to_logits
        

    
def multi_scale_logits(images,
                       model_options,
                       image_pyramid,
                       weight_decay=0.0001,
                       is_training=False,
                       fine_tune_batch_norm=False):
    """构建logits方法
    args:
        model_options: 网络配置的定义信息.
        image_pyramid: 图像金字塔
        weight_decay: 权重衰减
    """
    if not image_pyramid:
        image_pyramid=[1.0] # list
    # crop size
    crop_height=(
        model_options.crop_size[0]
        if model_options.crop_size else tf.shape(images)[1])
    crop_width=(
        model_options.crop_size[1]
        if model_options.crop_size else tf.shape(images)[0])
    # decoder_output_stride 是在decoder单元,提炼分割结果时候使用的 input/output的比
    logits_output_stride=(
        model_options.decoder_output_stride or model_options.output_stride)
    

    logit_height=cal_scaled_dim_val(crop_height,max(1.0,max(image_pyramid))/logits_output_stride) # 这个image_pyramid其实不是很清楚它的含义,是同尺寸的images组还是downsize后images
    logit_width=cal_scaled_dim_val(crop_width,max(1.0,max(image_pyramid))/logits_output_stride)   # 2018-09-17: 这个image_pyramide是包含了一组图像缩放的fractor.并不是图像本身.
    
    outputs_to_scales_to_logits={
        k:{}
        for k in model_options.outputs_to_num_classes
    }
    
    # step 1 对于每一个缩略图
    for image_scale in image_pyramid:
        if image_scale!=1.0:
            # 不是原图,需要缩放
            # 有了缩放因子,需要计算对应的缩放尺寸
            scaled_height=cal_scaled_dim_val(crop_height,image_scale)
            scaled_width=cal_scaled_dim_val(crop_width,image_scale)
            scaled_crop_size=[scaled_height,scaled_width]
            # 有了缩放尺寸,需要对原图做缩放了
            scaled_images=tf.image.resize_bilinear(images,scaled_crop_size,align_corners=True)
            
            if model_options.crops_size:
                scaled_images.set_shape([None,scaled_height,scale_width,3]) # 如果需要crop size的话,我们把scaled_images reshape成3个chn的.
        else:
            # 原图
            scaled_crop_size=model_options.crop_size
            scaled_images=images
        
        # 用做过scale的尺寸替换参数中的crop_size,然后生成网络
        updated_options=model_options._replace(crop_size=scaled_crop_size)
        outputs_to_logits=_get_logits(
            scaled_images,
            updated_options,
            weight_decay=weight_decay,
            reuse=tf.AUTO_REUSE,
            is_training=is_training,
            fine_tune_batch_norm=fine_tune_batch_norm)
        # 此时拿到结果.对结果做一个reshape,以便和其他的scale pyramid做融合使尺寸是合理的.
        for output in sorted(outputs_to_logits):
            outputs_to_logits[output]=tf.image.resize_bilinear(
                outputs_to_logits[output],
                [logit_height,logit_width],
                align_corners=True)
            
        # 只有一层pyramid,就可以返回
        if len(image_pyramid)==1:
            for output in sorted(model_options.outputs_to_num_classes):
                # 第k个scaler fractor对应的LOGITS_SCOPE_NAME,AKA,"logits"
                outputs_to_scales_to_logits[output][LOGITS_SCOPE_NAME]=outputs_to_logits[output]
            
            return outputs_to_scales_to_logits
        
        # 如果有多个pyramid fractor,需要按照对应的标签保存 
        for output in sorted(model_options.outputs_to_num_classes):
            outputs_to_scales_to_logits[output]['logits_%.2f'%image_scale]=outputs_to_logits[output]
            
    # 把多个pyramid fractor融合
    # 需要新创建一个维度,该维度为了融合使用
    for output in model_options.outputs_to_num_classes:
        all_logits=[
            tf.expand_dims(logits,axis=4)
            for logits in outputs_to_scales_to_logits[output].values()
        ]
        # 在这个新维度上做concat( 理解为连接)
        all_logits=tf.concat(all_logits,axis=4)
        # 根据不同的融合方法采用不同的tf的融合方法
        merge_fn=(
            tf.reduce_max
            if model_options.merge_method=='max' else tf.reduce_mean)
        # 在新增维度上融合.
        outputs_to_scales_to_logits[output][MERGED_LOGITS_SCOPE]=merge_fn(all_logits,axis=4)
    
    return outputs_to_scales_to_logits
    
                
    
            
            
    

 ![image](https://img-blog.csdn.net/20180518234043625?watermark/2/text/aHR0cHM6Ly9ibG9nLmNzZG4ubmV0L29KaU1vRGVZZTEyMzQ1/font/5a6L5L2T/fontsize/400/fill/I0JBQkFCMA==/dissolve/70)
    

![image](http://p0.ifengimg.com/pmop/2018/0326/34D889CEB77343C271F28FE97BBCF0CD5265946B_size25_w1080_h356.jpeg)

In [3]:
import numpy as np

In [4]:
x=np.linspace(1,200,100)

In [5]:
x

array([  1.        ,   3.01010101,   5.02020202,   7.03030303,
         9.04040404,  11.05050505,  13.06060606,  15.07070707,
        17.08080808,  19.09090909,  21.1010101 ,  23.11111111,
        25.12121212,  27.13131313,  29.14141414,  31.15151515,
        33.16161616,  35.17171717,  37.18181818,  39.19191919,
        41.2020202 ,  43.21212121,  45.22222222,  47.23232323,
        49.24242424,  51.25252525,  53.26262626,  55.27272727,
        57.28282828,  59.29292929,  61.3030303 ,  63.31313131,
        65.32323232,  67.33333333,  69.34343434,  71.35353535,
        73.36363636,  75.37373737,  77.38383838,  79.39393939,
        81.4040404 ,  83.41414141,  85.42424242,  87.43434343,
        89.44444444,  91.45454545,  93.46464646,  95.47474747,
        97.48484848,  99.49494949, 101.50505051, 103.51515152,
       105.52525253, 107.53535354, 109.54545455, 111.55555556,
       113.56565657, 115.57575758, 117.58585859, 119.5959596 ,
       121.60606061, 123.61616162, 125.62626263, 127.63

In [8]:
sess=tf.Session()
y=tf.reduce_mean(x)
print(sess.run(y))


100.5


In [22]:
#x1=tf.reshape(x,(5,5,5))
z= tf.constant([1, 2, 3, 4, 5, 6, 7,8])
sess.run(tf.global_variables_initializer())
print(sess.run(z))
z_r=tf.reshape(z,[2,2,2])
print(sess.run(z_r))
y_r=tf.reduce_mean(z_r,axis=[1,2])
print("reduce")
print(sess.run(y_r))
y0=tf.reduce_mean(z_r,axis=0)
print("reduce axis=0")
print(sess.run(y0))

y1=tf.reduce_mean(z_r,axis=1)
print("reduce axis=1")
print(sess.run(y1))


y2=tf.reduce_mean(z_r,axis=2)
print("reduce axis=2")
print(sess.run(y2))




[1 2 3 4 5 6 7 8]
[[[1 2]
  [3 4]]

 [[5 6]
  [7 8]]]
reduce
[2 6]
reduce axis=0
[[3 4]
 [5 6]]
reduce axis=1
[[2 3]
 [6 7]]
reduce axis=2
[[1 3]
 [5 7]]
